In [2]:
import pyodbc

# Replace with your actual credentials and server info
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 18 for SQL Server};'
    'SERVER=localhost;'
    'DATABASE=AdventureWorksLT2022;'
    'UID=sa;'
    'PWD=YourStrong:Passw0rd;'
    'TrustServerCertificate=yes;'
)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sys.databases")
for row in cursor.fetchall():
    print(row)

('master',)
('tempdb',)
('model',)
('msdb',)
('AdventureWorksLT2022',)


In [ ]:
sql = """
CREATE TABLE dbo.Customers (
    CustomerID UNIQUEIDENTIFIER PRIMARY KEY,
    CustomerName NVARCHAR(100),
    RegistrationDate DATETIME
)
"""

# Execute the SQL
cursor.execute(sql)
conn.commit()

print("Table created successfully.")


Table created successfully.


In [ ]:
sql = """
CREATE NONCLUSTERED INDEX IX_Customers_RegistrationDate ON dbo.Customers(RegistrationDate);
"""

# Execute the SQL
cursor.execute(sql)
conn.commit()

In [ ]:
sql = """
SELECT
    i.name AS IndexName,
    ps.avg_fragmentation_in_percent
FROM sys.dm_db_index_physical_stats (DB_ID(), OBJECT_ID('dbo.Customers'), NULL, NULL, 'SAMPLED') AS ps
JOIN sys.indexes AS i ON ps.object_id = i.object_id AND ps.index_id = i.index_id
WHERE i.name IS NOT NULL;
"""

cursor = conn.cursor()
cursor.execute(sql)
for row in cursor.fetchall():
    print(row)


In [ ]:
sql = """
INSERT INTO dbo.Customers (CustomerID, CustomerName, RegistrationDate)
SELECT TOP 10000 NEWID(), 'Customer ' + CAST(ROW_NUMBER() OVER (ORDER BY a.object_id) AS VARCHAR(10)), GETDATE() - (ROW_NUMBER() OVER (ORDER BY a.object_id) % 365)
FROM sys.all_objects a, sys.all_objects b;
"""

cursor = conn.cursor()
cursor.execute(sql)

In [ ]:
sql = """
SELECT
    i.name AS IndexName,
    ps.avg_fragmentation_in_percent
FROM sys.dm_db_index_physical_stats (DB_ID(), OBJECT_ID('dbo.Customers'), NULL, NULL, 'SAMPLED') AS ps
JOIN sys.indexes AS i ON ps.object_id = i.object_id AND ps.index_id = i.index_id
WHERE i.name IS NOT NULL;
"""

cursor = conn.cursor()
cursor.execute(sql)
for row in cursor.fetchall():
    print(row)

In [ ]:
#Fixing Fragmentation
## REORGANIZE (The "Lighter" Fix)

## ALTER INDEX IX_Customers_RegistrationDate ON dbo.Customers REORGANIZE;

sql = """
ALTER INDEX IX_Customers_RegistrationDate ON dbo.Customers REORGANIZE;
"""

cursor = conn.cursor()
cursor.execute(sql)


In [ ]:
sql = """
SELECT
    i.name AS IndexName,
    ps.avg_fragmentation_in_percent
FROM sys.dm_db_index_physical_stats (DB_ID(), OBJECT_ID('dbo.Customers'), NULL, NULL, 'SAMPLED') AS ps
JOIN sys.indexes AS i ON ps.object_id = i.object_id AND ps.index_id = i.index_id
WHERE i.name IS NOT NULL;
"""

cursor = conn.cursor()
cursor.execute(sql)
for row in cursor.fetchall():
    print(row)

In [ ]:
# REBUILD (The "Heavier" Fix)

# ALTER INDEX ALL ON dbo.Customers REBUILD;
GO

sql = """
ALTER INDEX ALL ON dbo.Customers REBUILD;
"""

cursor = conn.cursor()
cursor.execute(sql)


In [ ]:
sql = """
SELECT
    i.name AS IndexName,
    ps.avg_fragmentation_in_percent
FROM sys.dm_db_index_physical_stats (DB_ID(), OBJECT_ID('dbo.Customers'), NULL, NULL, 'SAMPLED') AS ps
JOIN sys.indexes AS i ON ps.object_id = i.object_id AND ps.index_id = i.index_id
WHERE i.name IS NOT NULL;
"""

cursor = conn.cursor()
cursor.execute(sql)
for row in cursor.fetchall():
    print(row)